<a href="https://colab.research.google.com/github/vishwanath090/Storygeneration/blob/main/storygeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy tensorflow keras nltk

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical
import nltk

# Download NLTK data
nltk.download('punkt')

# File paths
train_prompt_path = "/content/drive/MyDrive/writingPrompts/train.wp_source"
train_story_path = "/content/drive/MyDrive/writingPrompts/train.wp_target"
test_prompt_path = "/content/drive/MyDrive/writingPrompts/test.wp_source"
test_story_path = "/content/drive/MyDrive/writingPrompts/test.wp_target"
valid_prompt_path = "/content/drive/MyDrive/writingPrompts/valid.wp_source"
valid_story_path = "/content/drive/MyDrive/writingPrompts/valid.wp_target"

# Load data
def load_data(prompt_path, story_path):
    with open(prompt_path, 'r', encoding='utf-8') as f:
        prompts = f.readlines()
    with open(story_path, 'r', encoding='utf-8') as f:
        stories = f.readlines()
    return prompts, stories

train_prompts, train_stories = load_data(train_prompt_path, train_story_path)
test_prompts, test_stories = load_data(test_prompt_path, test_story_path)
valid_prompts, valid_stories = load_data(valid_prompt_path, valid_story_path)

# Tokenize and preprocess data
def preprocess_data(prompts, stories, max_len=100):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(prompts + stories)

    # Convert text to sequences
    prompt_sequences = tokenizer.texts_to_sequences(prompts)
    story_sequences = tokenizer.texts_to_sequences(stories)

    # Pad sequences
    prompt_sequences = pad_sequences(prompt_sequences, maxlen=max_len, padding='post')
    story_sequences = pad_sequences(story_sequences, maxlen=max_len+1, padding='post')

    return prompt_sequences, story_sequences, tokenizer

max_len = 100  # Maximum sequence length
train_prompt_seq, train_story_seq, tokenizer = preprocess_data(train_prompts, train_stories, max_len)
test_prompt_seq, test_story_seq, _ = preprocess_data(test_prompts, test_stories, max_len)
valid_prompt_seq, valid_story_seq, _ = preprocess_data(valid_prompts, valid_stories, max_len)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary size: {vocab_size}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Vocabulary size: 461227


In [ ]:
# Hyperparameters
embedding_dim = 256
latent_dim = 512
batch_size = 64
epochs = 10

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 256)      │    118,074,112 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 256)      │    118,074,112 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 512), (None,   │      1,574,912 │ embedding[0][0]        │
│                           │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 512),    │      1,574,912 │ embedding_1[0][0],     │
│                           │ (None, 512), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 512)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 461227)   │    236,609,451 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 475,907,499 (1.77 GB)

 Trainable params: 475,907,499 (1.77 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Prepare decoder input and output data
def prepare_decoder_data(story_sequences):
    decoder_input_data = story_sequences[:, :-1]
    decoder_output_data = story_sequences[:, 1:]
    return decoder_input_data, decoder_output_data

train_decoder_input, train_decoder_output = prepare_decoder_data(train_story_seq)
valid_decoder_input, valid_decoder_output = prepare_decoder_data(valid_story_seq)

# Convert decoder output to categorical (one-hot encoding)
train_decoder_output = to_categorical(train_decoder_output, num_classes=vocab_size)
valid_decoder_output = to_categorical(valid_decoder_output, num_classes=vocab_size)

# Train the model
history = model.fit(
    [train_prompt_seq, train_decoder_input],
    train_decoder_output,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([valid_prompt_seq, valid_decoder_input], valid_decoder_output)
)

In [ ]:
# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Function to generate stories
def generate_story(prompt_seq, max_len=100):
    states_value = encoder_model.predict(prompt_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']  # Start token

    stop_condition = False
    generated_story = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, '')
        generated_story += sampled_word + ' '

        if sampled_word == '<end>' or len(generated_story.split()) > max_len:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return generated_story

# Test the model
test_prompt = test_prompt_seq[0:1]  # Take the first test prompt
generated_story = generate_story(test_prompt)
print("Generated Story:")
print(generated_story)

In [ ]:
model.save("story_generation_model.h5")

In [ ]:
pip install streamlit

In [ ]:
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
model_path = "story_generation_model.h5"
model = load_model(model_path)

# Load the tokenizer (assuming you saved it during training)
import pickle
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Define the maximum sequence length
max_len = 100

# Encoder inference model
latent_dim = 512
encoder_inputs = model.input[0]  # Get the encoder input
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output  # Get the encoder states
encoder_model = tf.keras.Model(encoder_inputs, [state_h_enc, state_c_enc])

# Decoder inference model
decoder_inputs = model.input[1]  # Get the decoder input
decoder_state_input_h = tf.keras.Input(shape=(latent_dim,))
decoder_state_input_c = tf.keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[5]
decoder_embedding = model.layers[3]
decoder_dense = model.layers[6]

decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_embedding(decoder_inputs), initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Function to generate stories
def generate_story(prompt_text, max_len=100):
    # Tokenize and pad the input prompt
    prompt_seq = tokenizer.texts_to_sequences([prompt_text])
    prompt_seq = pad_sequences(prompt_seq, maxlen=max_len, padding='post')

    # Encode the input prompt
    states_value = encoder_model.predict(prompt_seq)

    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']

    stop_condition = False
    generated_story = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, '')
        generated_story += sampled_word + ' '

        # Exit condition: either max length is reached or the end token is predicted
        if sampled_word == '<end>' or len(generated_story.split()) > max_len:
            stop_condition = True

        # Update the target sequence and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return generated_story

# Streamlit app
st.title("AI-Based Story Generator")
st.write("Enter a writing prompt, and the AI will generate a story for you!")

# User input
prompt = st.text_input("Enter your writing prompt:", "A young wizard discovers a hidden library.")

# Generate story on button click
if st.button("Generate Story"):
    if prompt:
        with st.spinner("Generating story..."):
            story = generate_story(prompt)
        st.success("Story generated successfully!")
        st.write("**Generated Story:**")
        st.write(story)
    else:
        st.error("Please enter a writing prompt.")